# VinDsl.jl: Fast and furious statistical modeling
 
<img src="http://pearsonlab.github.io/images/plab_hex_icon_gray.png" width="150", align="left", style="float:left">

<br>
John Pearson  
Duke Institute for Brain Sciences



# About me

<img src="http://pearsonlab.github.io/images/plab_logo_dark.svg" width="400">

- computational neuroscience lab at Duke
- using Julia for about a year
- member of JuliaStats organization

# Our problem  
  
<figure style="display:inline-block;font-size:50%;float:left">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Monkey_grooming_02.JPG/640px-Monkey_grooming_02.JPG" width=400 align=left>
<figcaption>By AKS.9955, via Wikimedia Commons</figcaption>
</figure>

<figure style="display:inline-block;font-size:50%;float:right;text-align:center">
<img src="https://praneethnamburi.files.wordpress.com/2015/02/02_raster_baselineandstim.png" width=400 align=right>
<figcaption>https://praneethnamburi.wordpress.com/2015/02/05/simulating-neural-spike-trains/</figcaption>
</figure>


# Our problem

- model responses to known features (like GLM)
- infer latent features
- do this scalably for large populations of neurons
- use (generative) Bayesian models that account for uncertainty

# One solution: Sampling

- Markov Chain Monte Carlo (MCMC) methods come with guarantees about correctness
- lots of packages (Lora.jl, Mamba.jl, Stan.jl)
- **But** sampling does not scale well to millions of observations and parameters

# Another solution: The Max Power Way

<img src="max_power.jpg" width="600">

# Variational Bayesian Inference (VB)

Generative model for data: $p(y|\theta)p(\theta)$  
Actual posterior: $p(\theta|y) = p(y|\theta)p(\theta)/p(y)$  
Approximate posterior: $q(\theta)$  

Maximize **E**vidence **L**ower **Bo**und (ELBO) wrt $\theta$:

$$
\mathcal{L} = \log p(y) \ge -KL\left(q \middle\| p\right) = \mathbb{E}_q[\log p(y|\theta)] + \mathcal{H}[q]
$$

- $\mathbb{E}_q[\log p(y|\theta)]$ measures goodness of fit (data likely under approximate posterior)
- $\mathcal{H}[q]$ is the entropy (favors less certain models)

# Why VB?

- Scales well
- Can use well-studied optimization techniques

# Drawbacks:
- !@$*&# hard to code
- Can't quickly spec out a model like with Stan or JAGS/BUGS
- Traditionally, requires that distributions be conjugate, requires doing lots of algebra

# But VB is exploding!

- stochastic variational inference (SVI): [Hoffman et al.](http://dl.acm.org/citation.cfm?id=2502622)
- black box variational inference (BBVI): [Ranganath et al.](http://arxiv.org/abs/1401.0118)
- control variates: [Paisley et al.](http://arxiv.org/abs/1206.6430)
- local expectation gradients (LEG): [Titsias and L&aacute;zaro-Gredilla](http://papers.nips.cc/paper/5678-local-expectation-gradients-for-black-box-variational-inference)
- neural variational inference (NVIL): [Mnih and Gregor](http://arxiv.org/abs/1402.0030)
- variational autoencoders [Kingma and Welling](http://arxiv.org/abs/1312.6114), [Rezende et al.](http://arxiv.org/abs/1401.4082) 

# What's out there

- research code: individual algorithms, proof of concept 
- [Stan](http://mc-stan.org/) 
    - **Pros**: Robust, actively developed, good with stats, accessible from Julia
    - **Cons**: variational methods still experimental, C++
    

- [Edward](https://github.com/blei-lab/edward)
    - **Pros**: Python, multiple backends, from Stan and VB core developers
    - **Cons**: Python, very new


# What's out there
- [Theano](http://deeplearning.net/software/theano/) & [TensorFlow](https://www.tensorflow.org/)
    - **Pros**: well-tested, stable, well-engineered backends
    - **Cons**: complex, C++, not very stats-aware


# What do we want?
- write math, get code &mdash; a domain-specific language (DSL)
- easily generalize to different numbers of indices, structures
- only weakly opinionated about model structure or inference
- model code should be *hackable*
    - easy to use prefab pieces
    - not hard to write custom VB tricks
    - fast prototyping
- no (or minimal) algebra
- automatic gradients

Introducing...
![](http://www.joblo.com/newsimages1/vin.diesel_1920x1200_961)
## VinDsl.jl: Fast and furious variational inference

# Philosophy
- "internal" DSL
- loosely-coupled parts
- "consenting adults" outlook

# Today: two prototypes
- "Classical" models (conjugate + some optimization)
- ADVI "Black Box" models

# Model 1

### Finally, some code!

In [1]:
using Distributions
using VinDsl

# Model structure:
### Main idea: Factor graphs
- idea from Dahua Lin in [this talk](http://people.csail.mit.edu/dhlin/jubayes/julia_bayes_inference.pdf)
<img src="http://research.microsoft.com/en-us/um/people/cmbishop/prml/prmlfigs-png/Figure8.51.png" width="200">
- Nodes: arrays of distributions
- Factors $\leftrightarrow$ terms in variational objective
    - but not locked in to graphical model structure!

In [2]:
dims = (20, 6)

μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))

y[i, j] ~ Const(rand(dims));

# SHOW METHOD HERE

Nodes: under the hood
nodes define the q/approximate posterior/recognition model
~ defines a node
can use any distribution defined in the Distributions package
code parses the left and right-hand sides
indices on left get tracked and assigned to dimensions of parameter arrays
code is rewritten as a call to a node constructor

In [3]:
f = @factor LogNormalFactor y μ τ;

# SHOW METHOD HERE

In the future...

In [ ]:
@pmodel begin
    y ~ Normal(μ, τ)
end

New factor types can be defined with yet another macro:

In [ ]:
@deffactor LogNormalFactor [x, μ, τ] begin
    -(1/2) * ((E(τ) * ( V(x) + V(μ) + (E(x) - E(μ))^2 ) + log(2π) + Elog(τ)))
end

@deffactor LogGammaFactor [x, α, β] begin
    (E(α) - 1) * Elog(x) - E(β) * E(x) + E(α) * E(β) - Eloggamma(α)
end

- Uses a "mini-language" with `E(x)` $\equiv \mathbb{E}[X]$, `V(x)` $\equiv \textrm{cov}[X]$, etc.  
- Again, no need to track indices
    - multivariate distributions (Dirichlet, MvNormal) are automatically multivariate in these expressions
- `VinDsl` generates a `value(f)` function that handles indices appropriately and sums over the dimensions of the array

# Models are just factor graphs
Let's do a simple conjugate model:  
p-model:
$$
\begin{align}
\mu_j &\sim \mathcal{N}(0, 1) \\
\tau_j &\sim \mathrm{Gamma}(1.1, 1) \\
y_{ij} &\sim \mathcal{N}(\mu_j, \tau_j)
\end{align}
$$

q-model:
$$
\begin{align}
\mu &\sim \mathcal{N}(m, t) \\
\tau &\sim \mathrm{Gamma}(\alpha, \beta) \\
\end{align}
$$

In [4]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims))

# make factors
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior]);

# What's going on here?

In [5]:
update!(m)

In [6]:
m.update_strategy

Dict{VinDsl.Node,Symbol} with 7 entries:
  VinDsl.RandomNode{Distr… => :conjugate
  VinDsl.ConstantNode{Flo… => :constant
  VinDsl.RandomNode{Distr… => :conjugate
  VinDsl.ConstantNode{Flo… => :constant
  VinDsl.ConstantNode{Flo… => :constant
  VinDsl.ConstantNode{Flo… => :constant
  VinDsl.ConstantNode{Flo… => :constant

# Conjugacy
- right now `VinDsl` goes out of its way to handle conjugacy between nodes
- conjugate relationships not automatically detected, but easy to define
- `@defnaturals` returns expected sufficient statistics from a factor for a given target distribution

In [ ]:
@defnaturals LogNormalFactor μ Normal begin
    Ex, Eτ = E(x), E(τ)
    (Ex * Eτ, -Eτ/2)
end

# Index Bookkeeping
- nodes have associated indices
- factors know which indices go with which nodes, which indices to sum over
    - inner indices belong to, e.g., elements of a multivariate normal (should not be separated)
    - outer indices correspond to replicates of "atomic" variables  

So this is easy: `i` is inner:

In [9]:
d = 5
μ[i] ~ MvNormalCanon(zeros(d), diagm(ones(d)))
Λ[i, i] ~ Wishart(float(d), diagm(ones(d)));

But here, `i` is inner for $\mu$ but not for $\tau$. In any factor combining these two, $\tau$ will be treated like a vector because it matches an inner index for some node:

In [ ]:
μ[i] ~ MvNormalCanon(zeros(d), diagm(ones(d)))
τ[i] ~ Gamma(1.1 * ones(d), ones(d));

# Model 2: ADVI
[(Automatic Differentiation Variational Inference)](http://arxiv.org/abs/1603.00788)

# Two major ideas
1. We just need to define an ELBO (or *approxmate* an ELBO)
1. Any unimodal distribution is *approximately* a normal with constrained support

# Implementation
- Approximate ELBO by sampling from normal, transforming to constrained variables
- Let automatic differentiation handle the gradient calculation
- Do gradient ascent

# Hey, we have autodiff!
![http://www.juliadiff.org/](juliadiff.png)

# But there's always a problem:
- ForwardDiff defines `Dual <: Real`
- But Distributions.jl doesn't allow, e.g., `Dual` $\mu$ and $\sigma$ for `Normal`
- Then a lot of work...

# Today:
- Some distributions allow arbitrary parameter types:
    - using PDMats:
        - MvNormal, MvNormalCanon, GenericMvTDist
        - Wishart, InverseWishart
    - other:
        - Dirichlet, Normal, NormalCanon, Gamma, InverseGamma, Poisson

# This month:
- ~40 more distributions &mdash; almost all univariates
- PR in progress from @Halmoni100 in my lab
- some special functions in StatsFuns and Base still assume Float64

# Defining a simple model
$$\begin{align}
\sigma_\eta &\sim \mathrm{Gamma}(a_\eta, b_\eta) \\
a_u &\sim \mathcal{N}(\mu_a, \sigma_a) \\
\eta_{tu} &\sim \mathcal{N}(a_u, \sigma_\eta) \\
N_{tu} &\sim \mathrm{Poisson}(e^{\eta_{tu}}) \\
\end{align}$$

In [ ]:
function ELBO(x::Vector)
    ctr = 1
    L = zero(eltype(x))
    
    @advi_declarations begin
        a::Real()[U]
        σ::Positive()
        η::Real[T, U]    
    end
    
    @advi_model begin
        for u in 1:U
            a[u] ~ Normal(log(15.), 0.1)
        end
        S ~ InverseWishart(U, eye(U))
        for t in 1:T
            η[t] ~ MvNormal(a, S)
        end
        for t in 1:T
            for u in 1:U
                spikes[t, u] ~ Poisson(exp(η[t][u]))
            end
        end
    end
    
    L
end